In [110]:
import pandas as pd
import math
from google.cloud import storage


In [98]:
client = storage.Client(project="uk-election-406413")
bucket = client.bucket("ukelectiondata")
files = bucket.list_blobs()
file_names = [file.name for file in files]


In [99]:
election_results = bucket.blob(file_names[0])
election_results


<Blob: ukelectiondata, 1918-2019_election_results_by_constituency.xlsx, None>

In [100]:
# Commenting this out since I do not want to download the file every time I run the notebook,
# but if you want to trun locally youy will need to de comment it.

#election_results.download_to_filename('/Users/andreabrumana/code/willgreen93/UK_election/raw_data/1918-2019_election_results_by_constituency.xlsx')


In [111]:
# Need to install openpyxl. Run pip install openpyxl in terminal
data = pd.read_excel("/Users/andreabrumana/code/willgreen93/UK_election/raw_data/manual_cleaned_data.xlsx", sheet_name=None)


In [112]:
# Just want the data from 2001 onwards
years_in_scope = list(data.keys())[-6:]
years_in_scope


['2001', '2005', '2010', '2015', '2017', '2019']

In [113]:
data['2019'].head()


,Unnamed: 0,2019 GENERAL ELECTION,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47
0,NaN,Results by constituency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conservative,NaN,...,UUP,NaN,NaN,Alliance,NaN,NaN,Other,NaN,NaN,NaN
2,NaN,id,Constituency,County,Country/Region,Country,Electorate,NaN,Votes,Vote share,...,Votes,Vote share,NaN,Votes,Vote share,NaN,Votes,Votes Share,Total votes,Turnout
3,NaN,W07000049,ABERAVON,West Glamorgan,Wales,Wales,50750,NaN,6518,0.206279,...,NaN,NaN,NaN,NaN,NaN,NaN,731,0.023134,31598,0.622621
4,NaN,W07000058,ABERCONWY,Clwyd,Wales,Wales,44699,NaN,14687,0.460913,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31865,0.712879


In [128]:
def clean_data(data:dict, key:str):
    """A function that takes a dictionary of dataframes and a key and returns a cleaned dataframe"""

    # Access the dataframe
    temp_data = data[key]

    #Preparing the columns name
    two_rows = temp_data.iloc[1:3] # the columns name are in the second and third row of the temp_data dataframe
    new_list = []
    prev = None
    second_row = list(two_rows.iloc[1])

    for id, each in enumerate(list(two_rows.iloc[0])):
        if (not prev == "id")  and (isinstance(prev, str)):
            new_list.append(f"{prev}_{second_row[id]}")
        else:
            new_list.append(f"{each}_{second_row[id]}")
        prev = each

    cleaned_list = [element.lower().strip() for element in new_list]

    temp_data.columns = cleaned_list
    temp_data = temp_data.iloc[3:, 1:].reset_index(drop=True)
    temp_data.drop(columns='nan_nan', inplace=True)
    temp_data = temp_data.rename(columns=lambda x: x.replace('nan_', ''))
    temp_data = temp_data.rename(columns=lambda x: x.replace('_nan', ''))

    # There is an empty row, after the last row of the dataframe, and after it there is a legend of the data. Need to remove it
    empty_row_index = temp_data.index[temp_data.isnull().all(axis=1)].min()
    temp_data = temp_data.loc[:empty_row_index - 1, :]

    # Add an year column that depens on the key
    temp_data['year'] = int(key)
    temp_data = temp_data[["year", *temp_data.columns[:-1]]]

    # Clean data types
    votes_columns = [col for col in temp_data.columns if col.endswith('_votes')]
    share_columns = [col for col in temp_data.columns if col.endswith('share')]
    temp_data[votes_columns] = temp_data[votes_columns].astype(float)
    temp_data[share_columns] = temp_data[share_columns].astype(float)

    temp_data.fillna(0, inplace=True)

    # return temp_data, votes_columns, share_columns

    # Create homogenous columns in a new dataframe, and selecting just the columns we need
    clean_data = pd.DataFrame()

    clean_data['year'] = temp_data['year']
    clean_data['constituency_id'] = temp_data['id']
    clean_data['constituency'] = temp_data['constituency']
    clean_data['country'] = temp_data['country']
    clean_data['electorate'] = temp_data['electorate']
    clean_data['total_votes'] = temp_data['total votes']
    clean_data['tornout'] = temp_data['turnout']
    # Conservative
    clean_data['conservative_votes'] = temp_data['conservative_votes']
    clean_data['conservative_vote_share'] = temp_data['conservative_vote share']
    # Labour
    clean_data['labour_votes'] = temp_data['labour_votes']
    clean_data['labour_vote_share'] = temp_data['labour_vote share']
    # Liberal Democrats
    clean_data['liberal_democrats_votes'] = temp_data['liberal democrats_votes']
    clean_data['liberal_democrats_vote_share'] = temp_data['liberal democrats_vote share']
    # Main parties
    clean_data['main_parties_votes'] = clean_data['conservative_votes'] + clean_data['labour_votes'] + clean_data['liberal_democrats_votes']
    clean_data['main_parties_vote_share'] = clean_data['conservative_vote_share'] + clean_data['labour_vote_share'] + clean_data['liberal_democrats_vote_share']
    # Group all the parties that are not the main ones
    # clean_data['other_parties_votes'] = temp_data[votes_columns].sum(axis=1) - clean_data['main_parties_votes']
    # clean_data['other_parties_vote_share'] = temp_data[share_columns].sum(axis=1) - clean_data['main_parties_votes_share']
    # All parties
    # clean_data['total_votes_computed'] = temp_data[votes_columns].sum(axis=1)
    # clean_data['total_share'] = temp_data[share_columns].sum(axis=1)
    # Total votes

    return clean_data


In [130]:
# Testing the function for an year
test = clean_data(data, '2019')
#test['other_parties_votes'] = test[b].sum(axis=1)
test[test['constituency_id'] == 'W07000041']


,year,constituency_id,constituency,country,electorate,total_votes,tornout,conservative_votes,conservative_vote_share,labour_votes,labour_vote_share,liberal_democrats_votes,liberal_democrats_vote_share,main_parties_votes,main_parties_vote_share
647,2019,W07000041,YNYS MON,Wales,51925,36552,0.703938,12959.0,0.354536,10991.0,0.300695,0.0,0.0,23950.0,0.655231


In [131]:
# Applying the function to all the years in scope
cleaned_election_results = pd.concat([clean_data(data, key) for key in years_in_scope])
cleaned_election_results['other_parties_votes'] = cleaned_election_results['total_votes'] - cleaned_election_results['main_parties_votes']
cleaned_election_results['other_parties_vote_share'] = 1.0 - cleaned_election_results['main_parties_vote_share']
cleaned_election_results


,year,constituency_id,constituency,country,electorate,total_votes,tornout,conservative_votes,conservative_vote_share,labour_votes,labour_vote_share,liberal_democrats_votes,liberal_democrats_vote_share,main_parties_votes,main_parties_vote_share,other_parties_votes,other_parties_vote_share
0,2001,1,BEDFORD,England,67762,40579,0.598846,13297.0,0.327682,19454.0,0.479411,6425.0,0.158333,39176.0,0.965425,1403.0,0.034575
1,2001,2,LUTON NORTH,England,67554,39126,0.579181,12210.0,0.312069,22187.0,0.567065,3795.0,0.096994,38192.0,0.976128,934.0,0.023872
2,2001,3,LUTON SOUTH,England,71439,39351,0.550834,11586.0,0.294427,21719.0,0.551930,4292.0,0.109070,37597.0,0.955427,1754.0,0.044573
3,2001,4,MID BEDFORDSHIRE,England,70794,46638,0.658785,22109.0,0.474055,14043.0,0.301106,9205.0,0.197371,45357.0,0.972533,1281.0,0.027467
4,2001,5,NORTH EAST BEDFORDSHIRE,England,69877,45246,0.647509,22586.0,0.499182,14009.0,0.309619,7409.0,0.163749,44004.0,0.972550,1242.0,0.027450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,2019,E14001059,WYTHENSHAWE AND SALE EAST,England,76313,44759,0.586519,13459.0,0.300699,23855.0,0.532965,3111.0,0.069506,40425.0,0.903170,4334.0,0.096830
646,2019,E14001060,YEOVIL,England,82468,59260,0.718582,34588.0,0.583665,3761.0,0.063466,18407.0,0.310614,56756.0,0.957746,2504.0,0.042254
647,2019,W07000041,YNYS MON,Wales,51925,36552,0.703938,12959.0,0.354536,10991.0,0.300695,0.0,0.000000,23950.0,0.655231,12602.0,0.344769
648,2019,E14001061,YORK CENTRAL,England,74899,49505,0.660957,13767.0,0.278093,27312.0,0.551702,4149.0,0.083810,45228.0,0.913605,4277.0,0.086395


In [132]:
#Create a check columns to see if the total is 100%
cleaned_election_results['check'] = cleaned_election_results['conservative_vote_share']+cleaned_election_results['labour_vote_share']+cleaned_election_results['liberal_democrats_vote_share']+cleaned_election_results['other_parties_vote_share']
#Lets see which columsn have wrong data
cleaned_election_results[cleaned_election_results['check'] < 0.99]


,year,constituency_id,constituency,country,electorate,total_votes,tornout,conservative_votes,conservative_vote_share,labour_votes,labour_vote_share,liberal_democrats_votes,liberal_democrats_vote_share,main_parties_votes,main_parties_vote_share,other_parties_votes,other_parties_vote_share,check


In [133]:
# drop the check
cleaned_election_results.drop(columns='check', inplace=True)
# save to a csv file
cleaned_election_results.to_csv("/Users/andreabrumana/code/willgreen93/UK_election/raw_data/cleaned_election_results.csv", index=False)


In [134]:
# Upload the file to the bucket
cleaned_election_results_blob = bucket.blob("cleaned_election_results.csv")
cleaned_election_results_blob.upload_from_filename("/Users/andreabrumana/code/willgreen93/UK_election/raw_data/cleaned_election_results.csv")
#
